In [1]:
from rag_creator import RagCreator

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/shresthap/Library/Application Support/sagemaker/config.yaml


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# create an object to use openai
rag = RagCreator(llm_name='gpt-3.5-turbo')

Using openai llm


In [59]:
# load web
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
def load_web(url, limit=5):
    files_extension_to_exclude = ['.pdf', '.doc', '.docx', '.xls', '.xlsx',
                                  'jpg', '.jpeg', '.png', '.gif', '.ico',
                                  '.svg', 'webp']
    root = '/'
    nodes_list = deque()  # stack
    visited = set()  # set

    # add root
    nodes_list.append(root)

    # title and content only
    bs4_strainer = bs4.SoupStrainer(['h1', "article"])

    # set counter to 0
    counter = 0

    web_docs = []
    while len(nodes_list) > 0 and (counter < limit or limit == -1):
        counter = counter + 1
        node_str = nodes_list.pop()
        nodes = node_str.split('#')
        visited.add(nodes[0])
        full_node_url = url + nodes[0]
        print('Getting content from', full_node_url)
        loader = WebBaseLoader(
            web_paths=(full_node_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        loader.session.headers[
            'User-Agent'] = 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'

        # load and tokenize
        docs = loader.load_and_split(self.text_splitter)
        # to find all a tags
        scraped_doc = loader.scrape()
        # embed the docs
        web_docs.extend(docs)
        # update nodes_list using anchor tags
        for a in scraped_doc.findAll('a'):
            # make sure the href exists and is internal
            if not a.has_attr('href') or not a['href'].startswith('/'):
                continue
            # exclude files
            url_path = urlparse(a['href']).path
            file_extension = os.path.splitext(url_path)[1]
            if file_extension in files_extension_to_exclude:
                continue
            # maintain stack
            anchor_strs = a['href'].split('#')
            if anchor_strs[0] not in visited and anchor_strs[0] not in nodes_list:
                nodes_list.append(anchor_strs[0])
    return web_docs
    
docs = rag.load_web('https://www.niaid.nih.gov', 50)

Getting content from https://www.niaid.nih.gov/
Getting content from https://www.niaid.nih.gov/node/4220
Getting content from https://www.niaid.nih.gov/grants-contracts/preparing-foreign-organization-system-review
Getting content from https://www.niaid.nih.gov/node/4230
Getting content from https://www.niaid.nih.gov/grants-contracts/prepare-your-application
Getting content from https://www.niaid.nih.gov/grants-contracts/write-research-plan
Getting content from https://www.niaid.nih.gov/grants-contracts/plan-budget-personnel
Getting content from https://www.niaid.nih.gov/grants-contracts/salary-cap-stipends
Getting content from https://www.niaid.nih.gov/node/4855
Getting content from https://www.niaid.nih.gov/grants-contracts/contact-niaid-program-officer
Getting content from https://www.niaid.nih.gov/research/rules-policies-grants
Getting content from https://www.niaid.nih.gov/node/9847
Getting content from https://www.niaid.nih.gov/node/3769
Getting content from https://www.niaid.nih.

In [60]:
docs[0]

Document(metadata={'source': 'https://www.niaid.nih.gov/'}, page_content='Home Page\n\n\n\n\n\n\n\n\n\n\n\nVolunteer for Clinical Studies \n\nHealthy volunteers help NIAID to evaluate treatment and prevention options for a wide range of diseases and conditions in our mission areas. Help us to improve human health.\n\n\nVolunteer Now \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n                    RECOVER-TLC: The Next Phase in Long COVID Clinical Trials\n                     \nNIAID is leading the next stage of clinical trials to help Long COVID patients.\n\n \n\n\n\n\n\n\n\n \n\n\n                    NIAID Research Agenda for 2024 H5N1 Influenza\n                     \nNIAID has mobilized its intramural and extramural-supported research infrastructure to provide scientific support to the U.S. Department of Agriculture (USDA) and the Department of Health and Human Services (HHS).')

In [61]:
from langchain.vectorstores import DocArrayInMemorySearch

def create_vector_store(docs):
    # create a vector store
    return DocArrayInMemorySearch.from_documents(docs, embedding)
db  = create_vector_store(docs) 

In [62]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
# Build prompt
from langchain.prompts import PromptTemplate
template = """"You are an assistant for question-answering tasks specifically about the provided documents. "
    "Use ONLY the following pieces of retrieved context to answer the question. "
    "If you can't find the answer in the given context, say 'I'm sorry, but I couldn't find information about that in the provided context.' "
    "Do not use any external knowledge. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})
# create a chatbot chain. Memory is managed externally.
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    return_generated_question=True,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
)


In [65]:
qa_chain({"question": 'tell me about iron man?', "chat_history": []})

{'question': 'tell me about iron man?',
 'chat_history': [],
 'answer': "I'm sorry, but I couldn't find information about that in the provided context.",
 'source_documents': [Document(metadata={'source': 'https://www.niaid.nih.gov/node/2017'}, page_content='Visitor Information'),
  Document(metadata={'source': 'https://www.niaid.nih.gov/grants-contracts/write-research-plan'}, page_content="their team's expertise.Don't take a chance your reviewer will gloss over that one critical sentence buried somewhere in your Research Strategy or elsewhere. Write yourself an insurance policy against human fallibility: if it's a key point, repeat it, then repeat it again.Add more emphasis by putting the text in bold, or bold italics (in the modern age, we skip underlining—it's for typewriters).Here are more strategies from our successful PIs:While describing a method in the Approach section, they state their or collaborators' experience with it.They point out that they have access to a necessary pie

In [11]:
# docs count
rag.get_num_docs()

AttributeError: 'DocArrayInMemorySearch' object has no attribute '_collection'

In [12]:
qa_chain = rag.create_qa_chain('stuff', 4)

AttributeError: 'DocArrayInMemorySearch' object has no attribute '_collection'

In [10]:
qa_chain({"question": 'who is the narendra modi', "chat_history": []})

/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'question': 'who is the narendra modi',
 'chat_history': [],
 'answer': 'Narendra Modi is the Prime Minister of India, having held the position since May 2014. He is a member of the Bharatiya Janata Party (BJP) and the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist organization. Prior to becoming Prime Minister, Modi served as the Chief Minister of the Indian state of Gujarat from 2001 to 2014.',
 'source_documents': [Document(metadata={'source': 'https://www.niaid.nih.gov/node/4855'}, page_content="requirements. They are just a good way to build relationships with us and make sure your project is headed in the right direction. Before broaching the subject with your program officer, check with your business office about whether your institution will entertain a visit.Keep in Touch With NIAID StaffThough you rely on your institution's business office to pilot your grants management efforts, stay in contact with NIAID staff, too.Consult with your program officer on scientific ma

In [6]:
import gradio as gr
import time
# create ui
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    history = []
    def respond(message, chat_history):
        bot_message = qa_chain({"question": message, "chat_history": history})
        history.extend([(message, f"{bot_message['answer']}")])
        chat_history.append((message, f"{bot_message['answer']}, View more at: {bot_message['source_documents'][0].metadata['source']}"))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(inline=True)

/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/shresthap/Documents/Projects/rag/llama-langchain/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
